In [1]:
import bs4 as bs
import requests
import yfinance as yf
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 6)
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
from statsmodels.stats.diagnostic import het_arch
from statsmodels.stats.stattools import jarque_bera
from pykalman import KalmanFilter
import yfinance as yf
from math import floor

# ignore warnings
import warnings
warnings.filterwarnings('ignore')


In [2]:
# resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
# soup = bs.BeautifulSoup(resp.text, 'lxml')
# table = soup.find('table', {'class': 'wikitable sortable'})
# tickers = []
# for row in table.findAll('tr')[1:]:
#     ticker = row.findAll('td')[0].text
#     tickers.append(ticker)

# tickers = [s.replace('\n', '') for s in tickers]
# start = datetime.datetime(2019,1,1)
# end = datetime.datetime(2019,7,17)
# download price data for each ticker in the list
# data = yf.download(tickers, period='10y')
#save it in a csv file
# data.to_csv('s&p500.csv')

In [3]:
# # read data from csv file
# data = pd.read_csv('./data/s&p500.csv', header=[0, 1], index_col=0)
# data = data['Open']

In [4]:
# START_DATE = '2022-01-01'
# END_DATE = '2023-01-01'

# data = data[START_DATE:END_DATE]
# # get random 20 stocks from the universe
# # data2021 = data2021.sample(n=10, axis=1)

# # get ticker list of stocks that have missing data
# missing_data = data.isnull().sum()
# missing_data = missing_data[missing_data > 0]
# missing_data = missing_data.index.tolist()
# data = data.drop(missing_data, axis=1)



In [5]:
# import brute
# from tqdm import tqdm
# from multiprocessing import Pool
# import multiprocessing as mp

# # Function to write results to CSV
# def write_to_csv(results_dict, file_name):
#     with open(file_name, 'a') as f:
#         for key in results_dict.keys():
#             f.write("%s,%s,%s\n" % (key[0],key[1], results_dict[key]))

# # Loop through all pairs using multiprocessing
# d = {}
# stock_list = data.columns.tolist()

# # Prepare the list of pairs and corresponding data
# pair_data_list = []
# for i in range(len(stock_list)):
#     for j in range(i+1, len(stock_list)):
#         stock_pair = [missing_data[i], stock_list[j]]
#         pair_data = data[stock_pair]
#         pair_data_list.append((stock_pair, pair_data))

# # Use multiprocessing Pool to process the pairs concurrently
# batch_size = 2000

# with Pool(processes=2) as pool:
#     for i in range(0, len(pair_data_list), batch_size):
#         batch_data = pair_data_list[i:i + batch_size]
#         batch_results = list(tqdm(pool.imap_unordered(brute.calculate_sharpe, batch_data), total=len(batch_data)))
#         # print(batch_results)
#         write_to_csv(dict(batch_results), 'pairs.csv')



In [6]:

pairs = pd.read_csv('./data/ARIMA_PAIRS.csv', header=[0])
pairs.head()

,X,Y,sharpe_ratio
0,A,AAL,-0.578894
1,A,AAP,-0.500800
2,A,ABBV,-0.515649
3,A,AAPL,0.194942
4,A,ABC,1.229223


In [7]:
#count the number of pairs with sharpe ratio > 1.5
print('Number of pairs with sharpe ratio > 1.5: ', len(pairs[pairs['sharpe_ratio'] > 4]))

Number of pairs with sharpe ratio > 1.5:  20


In [8]:
# sort the pairs by absolute value of the sharpe ratio, ignoring the sign
pairs['abs_sharpe_ratio'] = pairs['sharpe_ratio'].abs()
pairs = pairs.sort_values(by=['abs_sharpe_ratio'], ascending=False)
pairs.head(500)

,X,Y,sharpe_ratio,abs_sharpe_ratio
114391,FOX,FOXA,5.979509,5.979509
98104,NWS,NWSA,5.911811,5.911811
71054,GOOG,GOOGL,5.097776,5.097776
104797,RSG,WM,4.872340,4.872340
117016,HWM,QRVO,4.599088,4.599088
...,...,...,...,...
2179,ABBV,ODFL,-3.022694,3.022694
111712,CFG,TYL,-3.022540,3.022540
71293,GOOG,VFC,3.022420,3.022420
55463,EBAY,MLM,3.021819,3.021819


In [10]:
# write the sorted pairs to a csv file
pairs.to_csv('./data/ARIMA_PAIRS_SORTED.csv', index=False)

In [9]:
pairs[pairs['abs_sharpe_ratio'] > 4 ]

,X,Y,sharpe_ratio,abs_sharpe_ratio
114391,FOX,FOXA,5.979509,5.979509
98104,NWS,NWSA,5.911811,5.911811
71054,GOOG,GOOGL,5.097776,5.097776
104797,RSG,WM,4.872340,4.872340
117016,HWM,QRVO,4.599088,4.599088
52545,DPZ,PPG,4.553498,4.553498
33190,CBOE,JNJ,4.475624,4.475624
65727,FFIV,MMC,4.417504,4.417504
120215,OGN,HSY,-4.332756,4.332756
83748,KIM,REG,4.254819,4.254819
